# Mapa de fuentes

**oct-nov 2023**

Actualizar el mapa con los lugares donde se tomaron las muestras de trabajo de campo (https://dila-ling.github.io/pict-glotales/mapa/). Este mapa tiene la locaclización de fuentes de texto y de audio. Queremos actualizarlo con la localización de las tomas de los nuevos audios incorporados en ago-sep 2023.

- mapa:[../pict-glotales/mapa.md](../pict-glotales/mapa.md)
- datos viejos: [glotales_geoloc](https://docs.google.com/spreadsheets/d/1q_HUGbLVsyo6P6ue9J26GH4UBmwz1G8b/edit#gid=2078261815)
- datos nuevos: [palabras_en_comun_audios](https://docs.google.com/spreadsheets/d/1GtliM_yutOBnWaferOmQjVNLt_xOxz56rst6ZR9MkXc/edit#gid=1308046217), Lengua, Comunidad, Latitud, Longitud

También quiero que el mapa tome los datos de un json o de un geojson. Ver [places-by-chapter-map](../notebooks/Del-Barco-Centenera/places-by-chapter-map.ipynb) para construir un [json](https://github.com/hdlabconicet/argentina-y-conquista-del-rio-de-la-plata/blob/7825d3a9f90f4225c322964d5acf4ba067e63b61/assets/js/mapData.js) de datos var places = {'PLACENAME': 'NombreComunidad', 'SOURCES': ['Gerzenstein, 1985', 'Gerzenstein, 1994', 'Gerzenstein, 1999', 'Tacconi, 2015'], 'COLOR': '#4363d8'(según lengua), 'LAT': -25.91667, 'LNG': -57.15}, ...]. [Mapa correspondiente](https://github.com/hdlabconicet/argentina-y-conquista-del-rio-de-la-plata/blob/7825d3a9f90f4225c322964d5acf4ba067e63b61/index.html). [geojson](../argentina-y-conquista-del-rio-de-la-plata/assets/js/mapData.geo.js) y [mapa correspondiente](https://github.com/hdlabconicet/argentina-y-conquista-del-rio-de-la-plata/blob/main/index.html)

Y ver [build-source-map](../notebooks/PICT-20161038/build-source-map.ipynb) para construcción de mapa actual.

Hacer un mapa donde los colores de los marcadores indiquen la lengua. 
Colores para marcadores: https://github.com/pointhi/leaflet-color-markers

En el popup info de nombre de comunidad y fuente

In [1]:
import pandas as pd
from json import dump, load,loads

## 1. Merge de fuentes nuevas y fuentes viejas

### 1.1 Datos de fuentes de audios

Acá están incluidos los datos de la tanda agosto/septiembre 2023 y los de 2019

In [3]:
sheet = "https://docs.google.com/spreadsheets/d/1GtliM_yutOBnWaferOmQjVNLt_xOxz56rst6ZR9MkXc/export?gid=1308046217&format=csv"
sheet = "palabras_en_comun_audios.csv"

df = pd.read_csv(sheet)
df.sample(5)

,Palabra,Lengua,Español,Recorte,Archivo,Formato,Inicio (sg),Fin (sg),Código,Cita Audio,...,Cita Bibliográfica,Herramienta,Transcriptorx,Tanda,Revisorx,Fuente,Consultante,Comunidad,Latitud,Longitud
116,jit͡ʃ'iniʃ,nivacle,hermano menor,2010_08_17_MV_AG_068_mihermanomenor,NaN,wav,NaN,NaN,NaN,NaN,...,"Gutiérrez, Analía (2010) Trabajo de campo sobr...",Word,AG,2023-08,AG,DILA inv,NaN,"Uj’e Lhavos, Filadelfia, Paraguay",-22.362072,-60.050070
4,ɬaʔa,chorote,fruta,ch_lha_a,BUCCA_A_02A-chorote,mp3,116.010337,116.77192,B1969,"Bucca, 1969",...,"Bucca, Salvador. “Elicitaciones en lenguas cho...",Audacity,NH,2019-06,AG,DILA repo,Manuel González,"Misión La Paz, Santa Victoria Este, Salta",-22.368465,-62.561983
6,ɬaj,wichi,fruta,wi_lhai,BUCCA_A_02A-wichi,mp3,42.729701,43.707936,B1969,"Bucca, 1969",...,"Bucca, Salvador. “Elicitaciones en lenguas cho...",Audacity,NH,2019-06,AG,DILA repo,Wilfredo Ibarra,"Misión La Paz, Santa Victoria Este, Salta",-22.368465,-62.561983
125,aβa̰tt͡ʃeʔeɬ,nivacle,ustedes,2011_07_12_FR_AG_ustedes,NaN,wav,NaN,NaN,NaN,NaN,...,"Gutiérrez, Analía (2010) Trabajo de campo sobr...",Word,AG,2023-08,AG,DILA inv,NaN,"Uj’e Lhavos, Filadelfia, Paraguay",-22.362072,-60.050070
69,saɁam,chorote,nosotros,CRT001R032I001_nosotros,CRT001R032I001,wav,1.870.280,1.871.546,G1968,"Gerzenstein, 1968",...,"Gerzenstein, Ana. (1968) ""Palabras en Chorote""...",Transcriber,PP,2023-08,NaN,AILLA,Manuel González,"Misión La Paz, Santa Victoria Este, Salta, 6 d...",-22.368465,-62.561983


In [ ]:
## Seleccionamos columnas
data_audios = df[['Cita Audio','Cita Bibliográfica','Lengua', 'Tanda','Comunidad', 'Latitud', 'Longitud']].drop_duplicates()
## Agregamos tipo de soporte
data_audios['Soporte'] = 'audio'
## Cambiamos nombre de columna y reordenamos para que coincida con el df de fuentes textuales
data_audios['Cita'] = data_audios['Cita Audio']
data_audios = data_audios[['Cita','Cita Bibliográfica','Lengua', 'Soporte','Comunidad', 'Latitud', 'Longitud']]
data_audios

In [ ]:
## Los nombres de comunidades tienen info redundante para el mapa (país y provincia)
## Los eliminamos
def normalizar_comunidad(s):
    return s.split(',')[0]

data_audios['Comunidad'] = data_audios['Comunidad'].map(normalizar_comunidad)
data_audios

In [ ]:
## Limpiamos 
data_audios = data_audios.dropna().drop_duplicates()
data_audios = data_audios.drop(41) # 39 y 41 son duplicados
data_audios

In [ ]:
print(f'Tenemos {len(data_audios)} registros para fuentes de audio')

### 1.2 Datos de fuentes textuales

In [ ]:
data_textos = pd.read_csv('../PICT-20161038/fuentes-geoloc.csv') # CAICYT

## Filtramos sólo los datos de fuentes textuales
data_textos = data_textos[ data_textos['Soporte'] == 'texto' ]

## Renombramos columnas para que coincida con el df de fuentes de audio
data_textos['Cita'] = data_textos['Cita'].str.replace('(', '').str.replace(')', '')
data_textos = data_textos.rename(columns={'Referencia':'Cita Bibliográfica', 'Comunidad, locación':'Comunidad'})
data_textos

In [ ]:
import re

## Normalizamos info de Comunidad
data_textos['Comunidad'] =  data_textos['Comunidad'].str.replace(' Formosa', '')
data_textos['Comunidad'] =  data_textos['Comunidad'].str.replace(' Chaco', '')
data_textos['Comunidad'] =  data_textos['Comunidad'].str.replace(', Bolivia', '')
data_textos['Comunidad'] =  data_textos['Comunidad'].str.replace(', Paraguay', '')
data_textos['Comunidad'] =  data_textos['Comunidad'].str.replace(', Asunción', '')
data_textos['Comunidad'] =  data_textos['Comunidad'].str.replace('el algarrobal', 'El Algarrobal')
data_textos['Comunidad'] =  data_textos['Comunidad'].str.replace('Misión la Paz', 'Misión La Paz')
data_textos['Comunidad'] =  data_textos['Comunidad'].str.replace('La gracia', 'La Gracia')
data_textos['Comunidad'] =  data_textos['Comunidad'].str.replace('colonia 22', 'Colonia 22')
data_textos['Comunidad'] =  data_textos['Comunidad'].str.replace('Santa rosa', 'Santa Rosa')
data_textos['Comunidad'] =  data_textos['Comunidad'].str.replace('La Estrella y la Bolsa', 'La Estrella y La Bolsa')
data_textos['Comunidad'] =  data_textos['Comunidad'].str.replace('Santa rosa', 'Santa Rosa')

def normalizar_comunidad(s):
    s = re.sub(r' \([^\)]+\)', '', s)
    return s

data_textos['Comunidad'] = data_textos['Comunidad'].map(normalizar_comunidad)

⚠️⚠️⚠️

PROBLEMA ASISTEMATICIDAD NOMBRES Y LOC DE COMUNIDADES

Misión Yacaré, Misión Pozo Yacaré, Misión Algarrobal, El Algarrobal, La Estrella y la Bolsa, La Bolsa

Misión chorote 1, Misión Chorote I parcela 42, Tartagal

Son la misma comunidad?

⚠️⚠️⚠️

In [ ]:
## Separamos info de coordenadas en 'Latitud' y 'Longitud'
def get_lat(s):
    return float(s.split(',')[0].strip())

def get_lon(s):
    return float(s.split(',')[1].strip())

data_textos['Latitud'] = data_textos['Localización'].map(get_lat)
data_textos['Longitud'] = data_textos['Localización'].map(get_lon)

In [ ]:
## Eliminamos columnas y reordenamos para que coincida con df de audios
data_textos = data_textos.drop(columns=['Localización', 'Código'])

data_textos = data_textos[['Cita', 'Cita Bibliográfica', 'Lengua', 'Soporte', 'Comunidad', 'Latitud', 'Longitud']]

In [ ]:
data_textos

In [ ]:
## Concatenamos df de audios con df de textos

loc_fuentes = pd.concat([data_textos, data_audios])
loc_fuentes = loc_fuentes.reset_index(drop=True)
loc_fuentes

In [ ]:
loc_fuentes.sort_values(by=['Comunidad', 'Latitud', 'Longitud'])[['Comunidad',
       'Latitud', 'Longitud', 'Cita', 'Cita Bibliográfica', 'Lengua', 'Soporte']].to_excel("glotales_geoloc.xlsx", index=False)

Esto fue copiado a [PICT_Glotales_DILA/Mataguayas/glotales_geoloc](https://docs.google.com/spreadsheets/d/1q_HUGbLVsyo6P6ue9J26GH4UBmwz1G8b/edit#gid=2078261815)

Revisado y corregido por MD y AG.

## 2. Trabajo con datos curados y corregidos a mano

In [4]:
corregido = "https://docs.google.com/spreadsheets/d/1q_HUGbLVsyo6P6ue9J26GH4UBmwz1G8b/export?gid=1861121093&format=csv"
corregido = "glotales_geoloc.csv" ## BELERIAND
loc_fuentes = pd.read_csv(corregido)
loc_fuentes.head(5)

,Comunidad,Latitud,Longitud,Cita,Cita Bibliográfica,Lengua,Soporte
0,Asunción,-25.296674,-57.680663,"Gerzenstein, 1985","Gerzenstein, Ana. (1985). Elicitación. Texas, ...",maká,audio
1,Asunción,-25.296674,-57.680663,"Gerzenstein, 1984","Gerzenstein, Ana. (1984). Elicitación. Texas, ...",maká,audio
2,Colonia 22,-22.311337,-60.230060,"Gerzenstein, 1983","Gerzenstein, Ana (1983). Lengua Chorote. Varie...",chorote,texto
3,Doctor Pedro P. Peña,-22.448409,-62.349916,"Seelwische, 2016","Seelwische, José (2016). Nuevo Diccionario Niv...",nivacle,texto
4,El Algarrobal,-22.191800,-63.637100,"Najlis, 1984","Najlis, Elena (1984).Fonología de la protoleng...",chorote,texto


In [5]:
loc_fuentes = loc_fuentes.drop_duplicates()
print(f"Tenemos {len(loc_fuentes)} puntos para mostrar en el mapa")
print(f"\t{len(loc_fuentes.drop_duplicates(subset=['Cita Bibliográfica']))} fuentes diferentes")
print(f"\t{len(loc_fuentes.drop_duplicates(subset=['Comunidad']))} comunidades diferentes")
print(f"\t{len(loc_fuentes.drop_duplicates(subset=['Lengua']))} lenguas o grupos de lenguas diferentes")

Tenemos 68 puntos para mostrar en el mapa
	30 fuentes diferentes
	33 comunidades diferentes
	6 lenguas o grupos de lenguas diferentes


### 2.2 Agrupar lenguas, soportes y fuentes por comunidad

#### Lengua

In [6]:
comunidad_lenguas = loc_fuentes.groupby(['Comunidad', 'Latitud', 'Longitud'])['Lengua'].apply(set).apply(sorted).apply(', '.join).reset_index()
comunidad_lenguas

,Comunidad,Latitud,Longitud,Lengua
0,Asunción,-25.296674,-57.680663,maká
1,Colonia 22,-22.311337,-60.230060,chorote
2,Doctor Pedro P. Peña,-22.448409,-62.349916,nivacle
3,El Algarrobal,-22.191800,-63.637100,chorote
4,Filadelfia,-22.359317,-60.034104,nivacle
5,Ing. Juárez,-23.898800,-61.854600,wichí
6,J.G.Bazán,-24.542200,-60.831200,wichí
7,La Bolsa,-23.296800,-62.144200,chorote
8,La Estrella y La Bolsa,-23.820195,-64.063242,chorote
9,La Gracia,-22.376080,-62.522148,chorote


Arreglamos los registros de Misión La Paz y Misión San Leonardo para que no haya repetidos y para que las lenguas aparezcan en orden alfabético:

In [ ]:
comunidad_lenguas.loc[[18]]

In [ ]:
comunidad_lenguas.loc[[21]]

In [ ]:
comunidad_lenguas.iat[18,3]

In [7]:
comunidad_lenguas.iat[18,3] = 'chorote, nivacle, wichí'
comunidad_lenguas.iat[21,3] = 'chorote, nivacle, wichí'

comunidad_lenguas

,Comunidad,Latitud,Longitud,Lengua
0,Asunción,-25.296674,-57.680663,maká
1,Colonia 22,-22.311337,-60.230060,chorote
2,Doctor Pedro P. Peña,-22.448409,-62.349916,nivacle
3,El Algarrobal,-22.191800,-63.637100,chorote
4,Filadelfia,-22.359317,-60.034104,nivacle
5,Ing. Juárez,-23.898800,-61.854600,wichí
6,J.G.Bazán,-24.542200,-60.831200,wichí
7,La Bolsa,-23.296800,-62.144200,chorote
8,La Estrella y La Bolsa,-23.820195,-64.063242,chorote
9,La Gracia,-22.376080,-62.522148,chorote


#### Soporte

In [8]:
comunidad_soporte = loc_fuentes.groupby(['Comunidad', 'Latitud', 'Longitud'])['Soporte'].apply(set).apply(', '.join).reset_index()
comunidad_soporte

,Comunidad,Latitud,Longitud,Soporte
0,Asunción,-25.296674,-57.680663,audio
1,Colonia 22,-22.311337,-60.230060,texto
2,Doctor Pedro P. Peña,-22.448409,-62.349916,texto
3,El Algarrobal,-22.191800,-63.637100,texto
4,Filadelfia,-22.359317,-60.034104,texto
5,Ing. Juárez,-23.898800,-61.854600,texto
6,J.G.Bazán,-24.542200,-60.831200,texto
7,La Bolsa,-23.296800,-62.144200,texto
8,La Estrella y La Bolsa,-23.820195,-64.063242,texto
9,La Gracia,-22.376080,-62.522148,texto


Sumamos los datos de lengua y de soporte en el mismo df:

In [9]:
merge = comunidad_lenguas.merge(comunidad_soporte, on=["Comunidad", "Latitud", "Longitud"])
merge

,Comunidad,Latitud,Longitud,Lengua,Soporte
0,Asunción,-25.296674,-57.680663,maká,audio
1,Colonia 22,-22.311337,-60.230060,chorote,texto
2,Doctor Pedro P. Peña,-22.448409,-62.349916,nivacle,texto
3,El Algarrobal,-22.191800,-63.637100,chorote,texto
4,Filadelfia,-22.359317,-60.034104,nivacle,texto
5,Ing. Juárez,-23.898800,-61.854600,wichí,texto
6,J.G.Bazán,-24.542200,-60.831200,wichí,texto
7,La Bolsa,-23.296800,-62.144200,chorote,texto
8,La Estrella y La Bolsa,-23.820195,-64.063242,chorote,texto
9,La Gracia,-22.376080,-62.522148,chorote,texto


#### Fuentes

In [10]:
comunidad_fuentes = loc_fuentes.groupby(['Comunidad', 'Latitud', 'Longitud'])['Cita'].apply('; '.join).reset_index()
comunidad_fuentes

,Comunidad,Latitud,Longitud,Cita
0,Asunción,-25.296674,-57.680663,"Gerzenstein, 1985; Gerzenstein, 1984"
1,Colonia 22,-22.311337,-60.230060,"Gerzenstein, 1983"
2,Doctor Pedro P. Peña,-22.448409,-62.349916,"Seelwische, 2016"
3,El Algarrobal,-22.191800,-63.637100,"Najlis, 1984; Gerzenstein, 1978 ; Gerzenstein,..."
4,Filadelfia,-22.359317,-60.034104,"Fabre, 2014"
5,Ing. Juárez,-23.898800,-61.854600,"Nercesian, 2014"
6,J.G.Bazán,-24.542200,-60.831200,"Nercesian, 2014"
7,La Bolsa,-23.296800,-62.144200,"Drayson, 2009"
8,La Estrella y La Bolsa,-23.820195,-64.063242,"Carol, 2014a; Carol, 2014b"
9,La Gracia,-22.376080,-62.522148,"Drayson, 2009"


Sumamos los datos de fuentes al df con los datos de lenguas y soportes agregados por comunidad:

In [11]:
merge2 = merge.merge(comunidad_fuentes, on=["Comunidad", "Latitud", "Longitud"] )
merge2

,Comunidad,Latitud,Longitud,Lengua,Soporte,Cita
0,Asunción,-25.296674,-57.680663,maká,audio,"Gerzenstein, 1985; Gerzenstein, 1984"
1,Colonia 22,-22.311337,-60.230060,chorote,texto,"Gerzenstein, 1983"
2,Doctor Pedro P. Peña,-22.448409,-62.349916,nivacle,texto,"Seelwische, 2016"
3,El Algarrobal,-22.191800,-63.637100,chorote,texto,"Najlis, 1984; Gerzenstein, 1978 ; Gerzenstein,..."
4,Filadelfia,-22.359317,-60.034104,nivacle,texto,"Fabre, 2014"
5,Ing. Juárez,-23.898800,-61.854600,wichí,texto,"Nercesian, 2014"
6,J.G.Bazán,-24.542200,-60.831200,wichí,texto,"Nercesian, 2014"
7,La Bolsa,-23.296800,-62.144200,chorote,texto,"Drayson, 2009"
8,La Estrella y La Bolsa,-23.820195,-64.063242,chorote,texto,"Carol, 2014a; Carol, 2014b"
9,La Gracia,-22.376080,-62.522148,chorote,texto,"Drayson, 2009"


In [ ]:
merge2.to_csv("glotales_geoloc_agregado.csv", index=False)

### 2.2 Mapping de colores por lengua

- marcadores de colores: https://github.com/pointhi/leaflet-color-markers

In [12]:
merge2.drop_duplicates(subset=['Lengua'])[['Lengua']]

,Lengua
0,maká
1,chorote
2,nivacle
5,wichí
18,"chorote, nivacle, wichí"
20,"chorote, wichí"


In [13]:
## https://github.com/pointhi/leaflet-color-markers

d = {"maká": "gold", ## amarillo
"chorote": "blue", ## azul
"nivacle": "red", ## rojo
"wichí": "green", ## verde
"chorote, nivacle, wichí": "grey", ## gris
"chorote, wichí": "violet" ## violeta
    }

merge2['Color'] = merge2['Lengua'].map(d)
merge2

,Comunidad,Latitud,Longitud,Lengua,Soporte,Cita,Color
0,Asunción,-25.296674,-57.680663,maká,audio,"Gerzenstein, 1985; Gerzenstein, 1984",gold
1,Colonia 22,-22.311337,-60.230060,chorote,texto,"Gerzenstein, 1983",blue
2,Doctor Pedro P. Peña,-22.448409,-62.349916,nivacle,texto,"Seelwische, 2016",red
3,El Algarrobal,-22.191800,-63.637100,chorote,texto,"Najlis, 1984; Gerzenstein, 1978 ; Gerzenstein,...",blue
4,Filadelfia,-22.359317,-60.034104,nivacle,texto,"Fabre, 2014",red
5,Ing. Juárez,-23.898800,-61.854600,wichí,texto,"Nercesian, 2014",green
6,J.G.Bazán,-24.542200,-60.831200,wichí,texto,"Nercesian, 2014",green
7,La Bolsa,-23.296800,-62.144200,chorote,texto,"Drayson, 2009",blue
8,La Estrella y La Bolsa,-23.820195,-64.063242,chorote,texto,"Carol, 2014a; Carol, 2014b",blue
9,La Gracia,-22.376080,-62.522148,chorote,texto,"Drayson, 2009",blue


In [ ]:
d

### Mapa provisorio

In [14]:
temp_json = merge2.to_json(orient='index')
parsed_temp = loads(temp_json)

d = {"maká": "#FFD326", ## amarillo
"chorote": "#2A81CB", ## azul
"nivacle": "#CB2B3E", ## rojo
"wichí": "#2AAD27", ## verde
"chorote, nivacle, wichí": "#7B7B7B", ## gris
"chorote, wichí": "#9C2BCB" ## violeta
    }

with open('mapData.js', 'w') as fo:
    fo.write(f'var langColor = {d}\n')
    
    fo.write('var fuentesLoc = [')
    
    for el in parsed_temp:
        fo.write(f'{parsed_temp[el]},\n')
    fo.write(']')

In [ ]:
with open('mapa-provisorio.html', 'w') as fo:
    fo.write('<div id="map" class="map leaflet-container mb-3" style="height: 500px; position:relative;"></div>\n')
    ## usamos el json de datos guardado anteriormente
    fo.write('<script src="{{ site.baseurl }}/assets/js/mapData.js" type="text/javascript"></script>\n')
    fo.write('<script>\n')
    fo.write('var map = L.map("map").setView([-24.5811123, -60.4287612], 6);\n')
    fo.write("L.tileLayer('https://api.mapbox.com/styles/v1/{id}/tiles/{z}/{x}/{y}?access_token={accessToken}', {\n")
    fo.write("attribution: 'Map data &copy; <a href=\"http://openstreetmap.org\">OpenStreetMap</a>, <a href=\"http://creativecommons.org/licenses/by-sa/2.0/\">CC-BY-SA</a>, Imagery © <a href=\"http://mapbox.com\">Mapbox</a>',")    
    fo.write("maxZoom: 18,\n      tileSize: 512,\n      zoomOffset: -1,\n      id: 'mapbox/streets-v11',\n")
    fo.write("accessToken: 'pk.eyJ1IjoiaGRjYWljeXQiLCJhIjoiY2s5MDc0NGQ3MjBudTNtbWh1NzFmM2VjZyJ9.MuGwJ2d1CmhzIg4M-TQJWA'\n")    
    fo.write("}).addTo(map);\n\n")
    
    fo.write("""
        const markerIcon = L.Icon.extend({
        options: {
        shadowUrl: 'https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.7/images/marker-shadow.png',
        iconSize: [25, 41],
        iconAnchor: [12, 41],
        popupAnchor: [1, -34],
        shadowSize: [41, 41]
    }
    });
    """)
    
    fo.write("for (place of fuentesLoc) {\n")
    fo.write("  var marker = L.marker([place['Latitud'], place['Longitud']], {icon: new markerIcon({iconUrl: `https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-2x-${place['Color']}.png`})}).addTo(map);\n")
    fo.write("  marker.bindPopup(`<h6>${place['Comunidad']}</h6><br>${place['Cita']}`);\n")
    fo.write('}\n\n')    
    
    fo.write("var legend = L.control({position: 'bottomleft'});\n")
    fo.write("legend.onAdd = function (map) {\n")
    fo.write("var div = L.DomUtil.create('div', 'info legend');\n")
    fo.write("var labels = [];\n")

    fo.write("for (var key of Object.keys(langColor)) {\n")
    fo.write('    labels.push(`<i style="background: ${langColor[key]}"></i> ${key}`);\n')
    fo.write("}\n")
    
    fo.write("div.innerHTML = labels.join('<br>');\n")
    fo.write("return div;\n")
    fo.write("};\n")
    
    fo.write("legend.addTo(map);\n")
    
    fo.write('</script>')

In [ ]:
! mv mapa-provisorio.html ../pict-glotales/_includes/mapa-fuentes.html

In [15]:
! mv mapData.js ../pict-glotales/assets/js/

QUEDA PENDIENTE :

- HACER LAYER DE TIPO DE FUENTE
- RESALTAR PUNTOS AL HACER HOVER SOBRE EL NOMBRE DE LA LENGUA EN LA LEYENDA
- AGREGAR UNA IMAGEN DEL LUGAR EN EL POPUP
- LINK A LA BIBLIO COMPLETA EN EL POPUP O MOSTRAR LA BIBLIO COMPLETA AL CLICKEAR EN UNA REFERENCIA

### 2.3 Crear un geojson para Leaflet PENDIENTE

```
{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-48.63806, -26.24333]},
 'properties': {'titles': 'São Francisco do Sul', 'annotations': 5},
 'uris': ['http://sws.geonames.org/3449112'],
 'titles': ['São Francisco do Sul'],
 'names': ['São Francisco',
  'Сан-Франсиску-ду-Сул',
  'San Francisco del Sur',
  '南圣弗朗西斯科'],
 'source_gazetteers': ['http://www.geonames.org'],
 'quotes': ['Ibiaza',
  'puerto de Ibiacá',
  'Ibiaza',
  'puerto de Ibiacà',
  'Ibiaza'],
 'comments': ['Refiere a San Francisco de Mbiaza']
}

```

## Doc de Leaflet
- crear mapa con geometrías. Ver https://leafletjs.com/examples/geojson/ y https://leafletjs.com/examples/choropleth/

- filtrar por tipo de soporte. Ver https://leafletjs.com/examples/layers-control/ y https://leafletjs.com/examples/choropleth/

## Estética

- cambiar tiles: http://leaflet-extras.github.io/leaflet-providers/preview/
